In [1]:
import requests
import pandas as pd
#import datetime
import time

def getData(ticker, metric, no_of_days):
    
    ticker = ticker + '.NS'
    current_time = int(round(time.time()))
    start_time = current_time - (no_of_days)*24*60*60
    
    
    resp = requests.get('https://query1.finance.yahoo.com/v8/finance/chart/'
                        +ticker+'?formatted=true&crumb=3fOAXVz3HoU&lang=en-IN&region=IN&period1='
                        +str(start_time)+'&period2='+str(current_time)
                        +'&interval=1d&events=div%7Csplit&corsDomain=in.finance.yahoo.com')
    
    
    result = resp.json()
    
    
    timestamps = result['chart']['result'][0]['timestamp']
    values = result['chart']['result'][0]['indicators']['quote'][0][metric]
    
    #len(values)
    #datetime.datetime.fromtimestamp(timestamps[len(timestamps) - 1])
    
    
    d = {'timestamp': timestamps, 'value': values}
    dataFrame = pd.DataFrame(data=d)
    
    return dataFrame

In [2]:
val = getData('SBIN', 'close', 350)

In [3]:
val['5SMA'] = val['value'].rolling(window = 25).mean()

In [4]:
window_length = 25
data = val
sma = data['value'].rolling(window = window_length).mean()
d = {'timestamp': data['timestamp'], 'sma': sma}
dataFrame = pd.DataFrame(data=d)

In [5]:
import numpy as np

In [6]:
data['std'] = data['value'].rolling(window = 20).std() * 2
data['sma20'] = data['value'].rolling(window = 20).mean()

In [7]:
data['lower'] = data['sma20'] - data['std']

In [8]:
data.tail()

,timestamp,value,5SMA,std,sma20,lower
232,1548128700,290.250000,297.085996,9.889493,297.734996,287.845503
233,1548215100,286.649994,296.841996,10.971469,297.359996,286.388527
234,1548301500,288.049988,296.335995,11.520819,297.154996,285.634177
235,1548387900,285.399994,295.989995,12.638121,296.684996,284.046875
236,1548669600,281.600006,295.577996,14.329919,295.969997,281.640078


In [9]:
def get_bollinger_bands(data, window_length, variation):
    sma = data['value'].rolling(window = window_length).mean()
    std = data['value'].rolling(window = window_length).std()

    d = {'timestamp': data['timestamp'], 'sma': sma, 'std': std}
    dataFrame = pd.DataFrame(data=d)
    
    dataFrame['upper_band'] = dataFrame['sma'] + variation*dataFrame['std']
    dataFrame['lower_band'] = dataFrame['sma'] - variation*dataFrame['std']
    
    return dataFrame

In [11]:
get_bollinger_bands(val, 20, 2).tail()

,timestamp,sma,std,upper_band,lower_band
232,1548128700,297.734996,4.944746,307.624489,287.845503
233,1548215100,297.359996,5.485734,308.331465,286.388527
234,1548301500,297.154996,5.760409,308.675814,285.634177
235,1548387900,296.684996,6.319061,309.323117,284.046875
236,1548669600,295.969997,7.164959,310.299915,281.640078


In [12]:
val.tail()

,timestamp,value,5SMA,std,sma20,lower
232,1548128700,290.250000,297.085996,9.889493,297.734996,287.845503
233,1548215100,286.649994,296.841996,10.971469,297.359996,286.388527
234,1548301500,288.049988,296.335995,11.520819,297.154996,285.634177
235,1548387900,285.399994,295.989995,12.638121,296.684996,284.046875
236,1548669600,281.600006,295.577996,14.329919,295.969997,281.640078


In [18]:
list(data.tail()['timestamp'])[0]

1548128700

In [26]:
from datetime import datetime
import pytz
ist = pytz.timezone("Asia/Kolkata")

In [30]:
datetime.fromtimestamp(list(data.tail()['timestamp'])[1] + 375*60,ist).strftime('%Y-%m-%d %H:%M:%S')

'2019-01-23 15:30:00'

In [35]:
datetime.fromtimestamp(list((data['timestamp'] + 375*60).tail())[1],ist).strftime('%Y-%m-%d %H:%M:%S') 

'2019-01-23 15:30:00'

In [37]:
import requests
import pandas as pd
#import datetime
import time

def getData(ticker, metric, no_of_days):
    
    ticker = ticker + str('.NS')
    current_time = int(round(time.time()))
    start_time = current_time - (no_of_days)*24*60*60
    
    
    resp = requests.get('https://query1.finance.yahoo.com/v8/finance/chart/'
                        +ticker+'?formatted=true&crumb=3fOAXVz3HoU&lang=en-IN&region=IN&period1='
                        +str(start_time)+'&period2='+str(current_time)
                        +'&interval=1d&events=div%7Csplit&corsDomain=in.finance.yahoo.com')
    
    
    result = resp.json()
    
    
    timestamps = result['chart']['result'][0]['timestamp']
    values = result['chart']['result'][0]['indicators']['quote'][0][metric]
    
    #len(values)
    #datetime.datetime.fromtimestamp(timestamps[len(timestamps) - 1])
    
    
    d = {'timestamp': timestamps, 'value': values}
    dataFrame = pd.DataFrame(data=d)
    
    dataFrame['timestamp'] = dataFrame['timestamp'] + 375*60
    
    return dataFrame

In [39]:
d = getData('MARUTI', 'close', 365)

In [40]:
d

,timestamp,value
0,1517220000,9630.700195
1,1517306400,9571.549805
2,1517392800,9509.700195
3,1517479200,9401.849609
4,1517565600,8990.599609
5,1517824800,9080.599609
6,1517911200,8984.599609
7,1517997600,8921.799805
8,1518084000,9030.700195
9,1518170400,8936.099609


In [41]:
datetime.fromtimestamp(list((d['timestamp']).tail())[1],ist).strftime('%Y-%m-%d %H:%M:%S') 

'2019-01-23 15:30:00'